In [2]:
# install packages
%pip install boto3
%pip install sagemaker
%pip install -U sagemaker
%pip install torchmetrics
# %jupyter notebook --NotebookApp.max_buffer_size=12000000000

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# cell 01
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sagemaker
import torchmetrics
import torch
from sagemaker import get_execution_role


sagemaker_session = sagemaker.Session()

role = 'AmazonSageMaker-ExecutionRole-20220714T204241'
# role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [4]:
# cell 02
training_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/training/multi_label_new.csv'
#test_data_uri = 's3://multibert-8b87f872-a3fe-4a19-a016-e02402275450/test'

In [5]:
from sagemaker.pytorch import PyTorch

In [6]:
# cell 05
pytorch_estimator = PyTorch('multibert.py',
                            instance_type='ml.g4dn.16xlarge',
                            instance_count=3,
                            role = role,
                            framework_version='1.11.0',
                            py_version='py38',
                            source_dir = 'code',
                            hyperparameters = {'epochs': 4, 'batch-size': 32, 'learning-rate': 2e-05})


In [8]:
# cell 07
pytorch_estimator.fit(training_data_uri)
#pytorch_estimator.fit({'train': training_data_uri,
#                       'test': test_data_uri})

2022-07-19 05:39:00 Starting - Starting the training job...
2022-07-19 05:39:17 Starting - Preparing the instances for trainingProfilerReport-1658209139: InProgress
.........
2022-07-19 05:41:00 Downloading - Downloading input data
2022-07-19 05:41:00 Training - Downloading the training image........................
2022-07-19 05:45:04 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-19 05:45:07,598 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-19 05:45:07,616 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-19 05:45:07,621 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-19 05:45:08,023 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requ

In [7]:
# cell 09
predictor = pytorch_estimator.deploy(instance_type='ml.c5.12xlarge',
                                     initial_instance_count=3)

ValueError: Estimator is not associated with a training job

In [ ]:
# cell 11
data = "I am a test."
response = predictor.predict(data)